# LangGraph MCP Agent - Quick Start

This notebook demonstrates how to use the modular LangGraph MCP agent package.

**Note:** This notebook uses the refactored `langgraph_agent` package with proper separation of concerns.

## 1. Setup and Configuration

**Important:** After running the install cell below, restart the kernel before proceeding.


In [13]:
# Install the local package in development mode
import sys
import subprocess

# Install from parent directory
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-e", ".."])
print("✓ Package installed. Please restart the kernel to use the package.")


✓ Package installed. Please restart the kernel to use the package.


In [14]:
# Restart the kernel to load the installed package
import IPython
IPython.Application.instance().kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

In [12]:
# Import the agent package
from langgraph_agent import get_config, initialize_agent
from langgraph_agent.utils import get_workspace_client, setup_mlflow_tracking

# Load configuration
config = get_config()

# Show current configuration
print(f"Model Endpoint: {config.model.endpoint_name}")
print(f"Databricks Profile: {config.databricks.profile}")
print(f"UC Model: {config.uc.full_model_name}")

ModuleNotFoundError: No module named 'langgraph_agent'

## 2. Test Authentication

In [3]:
from langgraph_agent.utils import verify_authentication

# Verify authentication
auth_info = verify_authentication(config.databricks.profile)
print(f"✓ Authenticated to: {auth_info['host']}")
print(f"✓ Profile: {auth_info['profile']}")
print(f"✓ Auth type: {auth_info['auth_type']}")

ModuleNotFoundError: No module named 'langgraph_agent'

## 3. Setup MLflow Tracking

In [ ]:
# Setup MLflow tracking
mlflow_config = setup_mlflow_tracking(
    profile=config.databricks.profile,
    experiment_name=config.mlflow.experiment_name,
    enable_autolog=True
)

print(f"✓ Tracking URI: {mlflow_config.get('tracking_uri')}")
print(f"✓ Experiment: {mlflow_config.get('experiment_name')}")

## 4. Initialize the Agent

In [ ]:
# Get workspace client
ws = get_workspace_client(config.databricks.profile)
ws_host = ws.config.host

# Build MCP server URLs
managed_urls = config.mcp.managed_urls or [
    f"{ws_host}/api/2.0/mcp/functions/system/ai"
]

print(f"MCP Servers: {managed_urls}")

# Initialize agent
agent = initialize_agent(
    workspace_client=ws,
    llm_endpoint_name=config.model.endpoint_name,
    system_prompt=config.model.system_prompt,
    managed_mcp_urls=managed_urls,
    custom_mcp_urls=config.mcp.custom_urls,
)

print("✓ Agent initialized successfully!")

## 5. Test the Agent

In [ ]:
# Test with a simple query
response = agent.predict({
    "input": [
        {"role": "user", "content": "What is 7*6 in Python?"}
    ]
})

print(response)

In [ ]:
# Test with streaming
print("Streaming response:")
print("-" * 60)

for chunk in agent.predict_stream({
    "input": [
        {"role": "user", "content": "Calculate the 15th Fibonacci number using Python"}
    ]
}):
    print(chunk)

## 6. Evaluate the Agent

In [ ]:
from langgraph_agent.evaluate import run_evaluation_pipeline

# Run evaluation
eval_results = run_evaluation_pipeline(
    agent=agent,
    dataset_path="data/eval_dataset.json",
    experiment_name=config.mlflow.experiment_name,
)

print(f"\nEvaluation Results:")
print(f"Dataset size: {eval_results['dataset_size']}")
print(f"Metrics: {eval_results['metrics']}")

## 7. Deploy to Unity Catalog

**Note:** This step requires the agent code to be in a Python module file.

In [ ]:
from langgraph_agent.deploy import full_deployment_pipeline

# Run full deployment pipeline
deployment_result = full_deployment_pipeline(
    config=config,
    validate=True,
)

print("\n✓ Deployment Complete!")
print(f"Model: {deployment_result['registered_model']['name']}")
print(f"Version: {deployment_result['registered_model']['version']}")
print(f"Deployment: {deployment_result['deployment']}")

## Next Steps

After deployment:
1. Test the agent in AI Playground
2. Share with SMEs for feedback
3. Monitor performance and traces
4. Iterate based on feedback

You can also use the CLI for these operations:
```bash
# Evaluate
langgraph-agent evaluate --dataset data/eval_dataset.json

# Deploy
langgraph-agent deploy

# Check configuration
langgraph-agent config-show
```